<a href="https://colab.research.google.com/github/AnnaFetz/AML_Challenge/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import os
import random
import time
import math
import tensorflow
import matplotlib.pyplot as plt
import cv2
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, Reshape, Activation, LeakyReLU, MaxPooling2D, UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras import backend as K


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
import sys 
from google.colab import drive 
import glob

path = "/content/drive" 
drive.mount(path, force_remount=True)  
data_path = "drive/MyDrive/challenge_AML/dataset" 

myfiles = glob.glob(os.path.join(data_path, '*')) 
print(myfiles)

Found GPU at: /device:GPU:0
Mounted at /content/drive
Mounted at /content/drive
['drive/MyDrive/challenge_AML/dataset/validation', 'drive/MyDrive/challenge_AML/dataset/training', 'drive/MyDrive/challenge_AML/dataset/test']


In [ ]:
class Dataset(object):
    def __init__(self, data_path):
        self.data_path = data_path
        assert os.path.exists(self.data_path), 'Insert a valid path!'

        # get class list
        self.data_classes = os.listdir(self.data_path)

        # init mapping dict
        self.data_mapping = {}

        # populate mapping dict
        for c, c_name in enumerate(self.data_classes):
            temp_path = os.path.join(self.data_path, c_name)
            temp_images = os.listdir(temp_path)

            for i in temp_images:
                img_tmp = os.path.join(temp_path, i)

                if img_tmp.endswith('.jpg'):
                    if c_name == 'distractor':
                        self.data_mapping[img_tmp] = -1
                    else:
                        self.data_mapping[img_tmp] = int(c_name)

        print('Loaded {:d} from {:s} images'.format(len(self.data_mapping.keys()),
                                                    self.data_path))

    def get_data_paths(self):
        # returns a list of imgpaths and related classes
        images = []
        classes = []
        for img_path in self.data_mapping.keys():
            if img_path.endswith('.jpg'):
                images.append(img_path)
                classes.append(self.data_mapping[img_path])
        return images, np.array(classes)


    def num_classes(self):
        # returns number of classes of the dataset
        return len(self.data_classes)


In [ ]:
validation_path = os.path.join(data_path, 'validation')
test_path= os.path.join(data_path, "test")
gallery_path = os.path.join(validation_path, 'gallery')
query_path = os.path.join(validation_path, 'query')
training_path = os.path.join(data_path, "training") 

training_dataset = Dataset(data_path = training_path)
gallery_dataset = Dataset(data_path=gallery_path)
query_dataset = Dataset(data_path=query_path)
test_dataset = Dataset(data_path = test_path)

# get training data and classes
training_paths, training_classes = training_dataset.get_data_paths()
test_paths, test_classes = test_dataset.get_data_paths()
# we get validation gallery and query data
gallery_paths, gallery_classes = gallery_dataset.get_data_paths()
query_paths, query_classes = query_dataset.get_data_paths()



Loaded 39218 from drive/MyDrive/challenge_AML/dataset/training images
Loaded 534 from drive/MyDrive/challenge_AML/dataset/validation/gallery images
Loaded 70 from drive/MyDrive/challenge_AML/dataset/validation/query images
Loaded 59 from drive/MyDrive/challenge_AML/dataset/test images


['4', 'distractor', '6', '8', '5', '9', '7', '3', '31', '2', '19', '16', '37', '35', '14', '18', '15', '13', '12', '1', '11', '10']


In [ ]:
datagen = ImageDataGenerator(featurewise_center=False, samplewise_center=True, 
                             featurewise_std_normalization=True, samplewise_std_normalization=True, 
                             rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, brightness_range=None, 
                             shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode="nearest", cval=0.0,
                             horizontal_flip=True, vertical_flip=True, rescale=None, preprocessing_function=None, data_format=None,
                             validation_split=0.4, dtype=None, 
)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
batch_size = 16


training_set = datagen.flow_from_directory(
    training_path,
     batch_size = batch_size,
     class_mode = 'input',
     subset = 'training',
     shuffle=True)


validation_set = datagen.flow_from_directory(
    training_path,
    target_size = (256, 256),
    batch_size = batch_size,
    class_mode = 'input',
    subset = 'validation',
    shuffle=False)



Found 23537 images belonging to 22 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(array([[[[ 0.6730305 ,  0.28892726,  0.10967909],
         [ 0.6730305 ,  0.28892726,  0.10967909],
         [ 0.6730305 ,  0.28892726,  0.10967909],
         ...,
         [ 0.69863737,  0.34014103,  0.10967909],
         [ 0.69863737,  0.34014103,  0.10967909],
         [ 0.69863737,  0.34014103,  0.10967909]],

        [[ 0.6730305 ,  0.28892726,  0.10967909],
         [ 0.6730305 ,  0.28892726,  0.10967909],
         [ 0.6730305 ,  0.28892726,  0.10967909],
         ...,
         [ 0.69863737,  0.34014103,  0.10967909],
         [ 0.69863737,  0.34014103,  0.10967909],
         [ 0.69863737,  0.34014103,  0.10967909]],

        [[ 0.6730305 ,  0.28892726,  0.10967909],
         [ 0.6730305 ,  0.28892726,  0.10967909],
         [ 0.6730305 ,  0.28892726,  0.10967909],
         ...,
         [ 0.69863737,  0.34014103,  0.10967909],
         [ 0.69863737,  0.34014103,  0.10967909],
         [ 0.69863737,  0.34014103,  0.10967909]],

        ...,

        [[ 0.6730305 ,  0.2377135 ,  

In [ ]:
input_model = Input(shape=(256, 256, 3))




In [ ]:
def mymodel1(input_img, TRAINABLE=False):

 # base_model = VGG16(include_top = False, weights='imagenet', input_shape=(256,256,3))

  #for layer in base_model.layers:
  #  layer.trainable=TRAINABLE
    
  # Define the autoencoder
  input_model = input_img
 
  # Encoder layers
  encoder = Conv2D(32, (3,3), padding='same', kernel_initializer='normal')(input_model)
  encoder = LeakyReLU()(encoder)
  encoder = BatchNormalization(axis=-1)(encoder)

  encoder = Conv2D(64, (3,3), padding='same', kernel_initializer='normal')(encoder)
  encoder = LeakyReLU()(encoder)
  encoder = BatchNormalization(axis=-1)(encoder)

  encoder = Conv2D(64, (3,3), padding='same', kernel_initializer='normal')(input_model)
  encoder = LeakyReLU()(encoder)
  encoder = BatchNormalization(axis=-1)(encoder)

  encoder_dim = K.int_shape(encoder)
  encoder = Flatten()(encoder)

  # Latent Space
  latent_space = Dense(16, name='latent_space')(encoder)
 

  # Decoder Layers
  decoder = Dense(np.prod(encoder_dim[1:]))(latent_space)
  decoder = Reshape((encoder_dim[1], encoder_dim[2], encoder_dim[3]))(decoder)

  decoder = Conv2DTranspose(64, (3,3), padding='same', kernel_initializer='normal')(decoder)
  decoder = LeakyReLU()(decoder)
  decoder = BatchNormalization(axis=-1)(decoder)

  decoder = Conv2DTranspose(64, (3,3), padding='same', kernel_initializer='normal')(decoder)
  decoder = LeakyReLU()(decoder)
  decoder = BatchNormalization(axis=-1)(decoder)

  decoder = Conv2DTranspose(32, (3,3), padding='same', kernel_initializer='normal')(decoder)
  decoder = LeakyReLU()(decoder)
  decoder = BatchNormalization(axis=-1)(decoder)

  decoder = Conv2DTranspose(3, (3, 3), padding="same")(decoder)
  output = Activation('sigmoid', name='decoder')(decoder)

  # Create model object
  autoencoder = Model(input_model, output, name='autoencoder')

  return autoencoder
  # Compile the model
  #autoencoder.compile(loss="mse", optimizer= Adam(learning_rate=1e-3))
  


autoencoder=mymodel1(input_model)
autoencoder.compile(loss='mean_squared_error', optimizer = Adam())
autoencoder.summary()


Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256, 256, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
flatten (Flatten)            (None, 4194304)           0         
_________________________________________________________________
latent_space (Dense)         (None, 16)                67108880  
_________________________________________________________________
dense (Dense)                (None, 4194304)           

In [ ]:
# Fit the model
from tensorflow.keras.callbacks import ModelCheckpoint

history = autoencoder.fit_generator(
          training_set,
          steps_per_epoch=training_set.n // batch_size*2,
          epochs=50,
          validation_data=validation_set,
          validation_steps=validation_set.n // batch_size,
          callbacks = [ModelCheckpoint('drive/MyDrive/challenge_AML/checkpoints', 
                                       monitor='val_loss', 
                                       verbose=0, 
                                       save_best_only=True)])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
1260/1471 [========================>.....] - ETA: 9:45 - loss: 0.7217

UnknownError: ignored

In [ ]:
latent_space_model = Model(
                      autoencoder.input, 
                      autoencoder.get_layer(‘latent_space’).output)

In [ ]:
# Load all images and predict them with the latent space model
def latent_space(path):

  X = []
  indices = []

  for i in tqdm(range(len(path)))):
    try:
      img_name = training_paths[i]
      img = load_img(os.path.join(path,img_name)), 
                    target_size = (256, 256))
      img = img_to_array(img) / 255.0
      img = np.expand_dims(img, axis=0)
      pred = latent_space_model.predict(img)
      pred = np.resize(pred, (16))
      X.append(pred)
      indices.append(img_name)

    except Exception as e:
      print(img_name)
      print(e)
          # Export the embeddings
  embeddings = {'indices': indices, 'features': np.array(X)}
  pickle.dump(embeddings, 
              open('drive/MyDrive/challenge_AML/pick.pickle', 'wb'))

  return X, indices



In [ ]:
query_array, query_img = latent_space(query_paths)
gallery_array, gallery_img = latent_space(gallery_paths)

def eucledian_distance(x,y):
  eucl_dist = np.linalg.norm(x - y)
  return eucl_dist

def top_images() :
  
  distances = {}
  for query_array, query_img in latent_space(query_paths):
    distances[query_img] = []
    for gallery_array, gallery_img in latent_space(gallery_paths):
      distances = eucledian_distance(query_array,gallery_array)
      distances[query_img].append((distances, gallery_img))

  for key in distances: 
    distances[key].sort()
    distances[key] = distances[key][:5]
  return distances



In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np

class FeatureExtractor:
    def __init__(self):
        # Use VGG-16 as the architecture and ImageNet for the weight
        base_model = VGG16(weights='imagenet')
        # Customize the model to return features from fully-connected layer
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    
    def extract(self, img):
        # Resize the image
        img = img.resize((224, 224))
        # Convert the image color space
        img = img.convert('RGB')
        # Reformat the image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # Extract Features
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)

DA RICORDARSI !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
checkpoint_path = "drive/MyDrive/challenge_AML/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
'''model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training'''

In [ ]:
# OUR TRAINING DATASET
from sklearn.utils import shuffle

train_images = preprocess_img(training_paths) 
autoencoder_train = autoencoder.fit(train_images,train_images, batch_size=32,epochs=50,verbose=1)






Epoch 1/50
22/22 [==============================] - 107s 2s/step - loss: 1038545.9742
Epoch 2/50
22/22 [==============================] - 31s 1s/step - loss: 5494.7614
Epoch 3/50
22/22 [==============================] - 31s 1s/step - loss: 5503.0977
Epoch 4/50
22/22 [==============================] - 31s 1s/step - loss: 5511.5277
Epoch 5/50
22/22 [==============================] - 31s 1s/step - loss: 5469.8775
Epoch 6/50
22/22 [==============================] - 31s 1s/step - loss: 5597.3423
Epoch 7/50
22/22 [==============================] - 31s 1s/step - loss: 5526.7040
Epoch 8/50
22/22 [==============================] - 31s 1s/step - loss: 5576.5466
Epoch 9/50
22/22 [==============================] - 31s 1s/step - loss: 5564.7479
Epoch 10/50
22/22 [==============================] - 31s 1s/step - loss: 5431.3470
Epoch 11/50
22/22 [==============================] - 31s 1s/step - loss: 5408.3149
Epoch 12/50
22/22 [==============================] - 31s 1s/step - loss: 5538.7516
Epoch 13/

In [ ]:
#30 epochs good

In [ ]:
query_images = extract_features(query_paths, autoencoder)

gallery_images = extract_features(gallery_paths,  autoencoder)


ValueError: ignored

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint_path = "drive/MyDrive/challenge_AML/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
'''model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training'''

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [ ]:
def preprocess_img(training_paths):
  input_shape = (224, 224, 3)
  prep_img=[]
  for n in range(len(training_paths)):
      img = image.load_img(training_paths[n],
                          target_size=(input_shape[0], input_shape[1]))
      img_array = image.img_to_array(img) #convert images in 3D arrays authomatically -> must use flatten() to push it into one dimension
      expanded_img_array = np.expand_dims(img_array, axis=0) #img_array has shape (None, 3) now; the first dimension need to be added through this command (tf)
      preprocessed_img = preprocess_input(expanded_img_array) #images are converted from RGB to BGR (tf)
      prep_img.append(preprocessed_img)

  prep_img = np.array(prep_img) 
  return prep_img

def extract_features(paths, model) :
  nf=[]
  np_array = preprocess_img(paths)

  for img in np_array:
    features_ppr_image = model.predict(img) 
    flattened_features = features_ppr_image.flatten() 
    normalized_ft = flattened_features / norm(flattened_features) #normalize
    nf.append(normalized_ft)

  return nf

In [ ]:
def augmented_img(training_paths):

      input_shape = (224, 224, 3)
      prep_img=[]
      img_in_folder=os.listdir(training_paths) #list of images in the folder considered
      for el in img_in_folder: #for each image in the considered folder
        path=os.path.join(training_paths, el)
        print(path)
        if path.endswith('jpg'):
          img = image.load_img(path, target_size=(input_shape[0], input_shape[1])) #read single image

          img_array = image.img_to_array(img) #convert images in 3D arrays authomatically -> must use flatten() to push it into one dimension
          expanded_img_array = np.expand_dims(img_array, axis=0) #img_array has shape (None, 3) now; the first dimension need to be added through this command (tf)
          preprocessed_img = preprocess_input(expanded_img_array) #images are converted from RGB to BGR (tf)
          prep_img.append(preprocessed_img)

      for img in prep_img:

        i = 0
        for batch in datagen.flow(prep_img, 
                                    batch_size=64, #16
                                    save_to_dir=training_paths, 
                                    save_prefix='aug',
                                    save_format='jpg'):    
          i += 1    
          if i > 5:     
            break

list_classes = set(training_classes)
list_classes.remove(-1)
list_classes.add('distractor')
list_classes = list(list_classes)
for el in list_classes:
    folder = os.path.join(training_path, str(el))
    #print(folder)
    #img_in_folder=os.listdir(folder)
    augmented_img(folder)